In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import keras_tuner as kt

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [5]:
input_shape = (32, 32, 3) # CIFAR-10 images are 32x32 with 3 color channels
num_classes = 10 # There are 10 classification (car,truck,bike...)

In [6]:
""" # CNN Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    # CNN Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # CNN Layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Flattening layer
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128, activation='relu'))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model"""

" # CNN Layer 1\n    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))\n    model.add(MaxPooling2D((2, 2)))\n\n    # CNN Layer 2\n    model.add(Conv2D(64, (3, 3), activation='relu'))\n    model.add(MaxPooling2D((2, 2)))\n\n    # CNN Layer 3\n    model.add(Conv2D(128, (3, 3), activation='relu'))\n    model.add(MaxPooling2D((2, 2)))\n\n    # Flattening layer\n    model.add(Flatten())\n\n    # Fully connected layer\n    model.add(Dense(128, activation='relu'))\n\n    # Output layer\n    model.add(Dense(num_classes, activation='softmax'))\n\n    return model"

In [7]:
def build_cnn(hp):
    model = Sequential()

    # Add Convolutional layers with hyperparameter tuning
    for i in range(hp.Int('conv_layers', 1, 3)):  # Number of convolutional layers
        model.add(Conv2D(
            hp.Choice(f'filters_{i}', values=[32, 64, 128]),
            (3, 3),
            activation='relu',
            input_shape=input_shape if i == 0 else None
        ))
        model.add(MaxPooling2D((2, 2)))

    # Flattening layer
    model.add(Flatten())

    # Fully connected layer with hyperparameter tuning
    model.add(Dense(
        hp.Int('dense_units', min_value=64, max_value=256, step=64),
        activation='relu'
    ))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    compile_model(model, hp)

    return model

In [8]:
def compile_model(model, hp):
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'sgd']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

In [9]:
#epochs = 1000
#batch_size = 12

In [10]:
def train_model(model, x_train, y_train, x_test, y_test):


    history = model.fit(
        x_train, y_train,
        epochs=10,
        batch_size=12,
        validation_data=(x_test, y_test),

    )
    return history

In [11]:
def model_summary(model):
  model.summary()

In [12]:
def evaluate_model(model,x_test,y_test):
  loss,acc = model.evaluate(x_test,y_test)
  print('TESTING ACCURACY',acc)
  print('TESTING LOSS',loss)

In [13]:
def plot_accuracy(history):
    plt.figure(figsize=(12, 6))

    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

In [14]:
def hyperparameter_tuning(x_train, y_train, x_test, y_test):
    # Define the Tuner
    tuner = kt.Hyperband(
        build_cnn,
        objective='val_accuracy',
        max_epochs=10,
        hyperband_iterations=2,
        directory='my_dir',
        project_name='cifar10_tuning'
    )
    tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

  # Retrieve the best model
    best_model = tuner.get_best_models(num_models=1)[0]

  # Evaluate the best model
    loss, acc = best_model.evaluate(x_test, y_test, verbose=2)
    print('Best model accuracy:', acc)
    print('Best model loss:', loss)

    # Plot the training and validation accuracy of the best model
    history = tuner.get_best_hyperparameters()[0].get('history')
    plot_accuracy(history)

    # Summary of the best model
    best_model.summary()

In [15]:
hyperparameter_tuning(x_train, y_train, x_test, y_test)

Trial 60 Complete [00h 01m 09s]
val_accuracy: 0.6604999899864197

Best val_accuracy So Far: 0.7199000120162964
Total elapsed time: 00h 33m 46s
313/313 - 1s - 4ms/step - accuracy: 0.7199 - loss: 1.1083
Best model accuracy: 0.7199000120162964
Best model loss: 1.1083009243011475


KeyError: 'history does not exist.'